# Hiearchical Vector Concatenation

In this notebook, we are going to simulate the concatenation of two vectors using the HVC technique

### defining functions

In [1]:
import pandas as pd
import random
from itertools import combinations, product
from scipy.spatial import distance
import numpy as np
from sklearn.neighbors import NearestNeighbors
import json
import math

def vector_(vector, new_vector_length, x):
    vector = [(vector[i], vector[i]*x)[i < new_vector_length] for i in range(len(vector))]
    vector = np.array(vector)
    return vector

def test_knn(df, class_n, minimum_scale_coefficient, test_amount, sample_id=None, minimum_sample_n=0, verbose=False):
    '''
    tests each individual class sample vs all the other sample in the dataset
    when minimum_scale_coefficient==-1 means calculation cannot be done: WE DO NOT RAISE ERROR
    '''
    if minimum_scale_coefficient == -1:
        if verbose:
            print('minimum_scale_coefficient == -1')
            print('inconsistencies:', -1)
        return True

    class Found(Exception): pass
    # df = pd.read_parquet(f'{exp}/df_hierarchical_vector.parquet') #we reset the vectors so they do not kee scaling per iteration
    df_ = df.copy()

    # test for inefficiencies
    key1 = class_n
    if sample_id:
        samples = [sample_id]
    else:
        samples = [*df_[df_['class_1']==key1].index]

    for r in samples:
        if r >= minimum_sample_n:

            if verbose : print('**', '\tsample:', r, '\tmsc:', minimum_scale_coefficient)
            
            for msc_variation in [['less_than', minimum_scale_coefficient-test_amount], ['more_than', minimum_scale_coefficient+test_amount]]:
                
                # df = pd.read_parquet(f'{exp}/df_hierarchical_vector.parquet') #we reset the vectors so they do not keep scaling per iteration
                df_ = df.copy()
                df_['vector_full'] = df_['vector_full'].apply(lambda x : vector_(x, new_vector_length, msc_variation[1]))

                # classes
                nbrs = NearestNeighbors(n_neighbors=len(df_), metric='euclidean').fit(df_['vector_full'].values.tolist())
                distances, indices = nbrs.kneighbors([df_['vector_full'].iloc[r]]) # EDIT VECTOR 0 HERE

                list1 = [df_.iloc[x]['class_1'].values.tolist() for x in indices][0]
                counter = 1
                for i in range(len(list1)-1):
                    # count contaminations
                    el = list1[i], list1[i+1]
                    if el[0] != el[1]:
                        # print(r, indices[0][i:i+1], list1[i:i+1], distance.euclidean(df_.iloc[indices[0][0]]['vector_full'], df_.iloc[indices[0][i]]['vector_full']))
                        counter += 1

                inconsistencies = counter-n_classes
                if msc_variation[0] == 'less_than':
                    # should have inconsistencies, because we test it on a scaler < msc+test_amount

                    if inconsistencies == 0:
                        if verbose : 
                            print('<', 'ERR')
                            print('<', 'inconsistencies:', inconsistencies)
                        # raise Found
                        return False
                    else:
                        if verbose :
                            print('<', 'inconsistencies:', inconsistencies)
                        return True
                        
                if msc_variation[0] == 'more_than':
                    # should not have inconsistencies, because we test it on a scaler > msc+test_amount

                    if inconsistencies == 0:
                        if verbose : 
                            print('>', 'inconsistencies:', inconsistencies)
                        return True
                        
                    else:
                        if verbose :
                            print('>', 'ERR')
                            print('>', 'inconsistencies:', inconsistencies)
                        return False
                        # raise Found
                        
        if sample_id is not None:
            return True
            
def triangular_extension(A, B, C, concat_vector_length):
    
    if not len(A)==len(B)==len(C):
        raise Exception('all vectors must be of the same length')

    n = len(A)
    m = concat_vector_length
    numerator = 0
    for i in range(m, n):
        numerator += (2*A[i]*(B[i]-C[i]) + C[i]**2 - B[i]**2)

    denominator = 0
    for i in range(m):
        denominator += ( 2*A[i]*(C[i]-B[i]) + B[i]**2 - C[i]**2 )
    
    try:
        # AB will never be larger than AC
        # issue 1: negative quotient (sqrt is impossible)
        # issue 2: denominator is 0
        total = numerator/denominator
        total = math.sqrt(total)
        return total   
    except:
        # AB can be larger than AC
        return -1
    
def get_msc(df_total, sample_id, verbose=False):
    """
    we get the exact minimum scale coefficient given a sample.
    in total, a given A, B, C will have the same samples with an msc score for each step of a single path
    for a given A, we need to get the highest msc of the sample for it to work
    """
    df_ = df_total[(df_total['index']==sample_id)]
    if verbose :
        display(df_)
    return df_.minimum_scale_coefficient.max()

letters = 'abcdefghijklmnopqrstwxyz'
letters = letters.upper()
def create_class(k):
    class_name = ''.join(random.choices(letters, k=k))
    return class_name

### creating sample dataset

In [4]:
#   non-labeled df
initial_vector_n = 700 # 1h computation
initial_vector_length = 100
df = pd.DataFrame([[[random.random()*2-1 for x in range(initial_vector_length)] for k in range(initial_vector_n)]]).T
df.columns = ['vector_0']
df['vector_full'] = df['vector_0'] 
df

#   labeled df
for depth in range(1):
    # we start counting from 1, the previous vector start from depth 0
    depth += 1
    n_classes = 8
    new_vector_length = 100

    # the vector for each class has to be identical
    classes = [(str(x), '0'+str(x))[x<10] for x in range(n_classes)]
    # classes = ['A', 'B', 'C', 'D']
    arr1 = {key:[random.random()*2-1 for x in range(new_vector_length)] for key in classes}
    df_new = pd.DataFrame([[x, arr1[x]] for x in random.choices(classes, k=len(df))])
    df_new.columns = [f'class_{depth}', f'vector_{depth}']
    df_new_clean = df_new.drop_duplicates(f'class_{depth}')

    df = pd.concat([df_new, df], axis=1)

    # for now we just sum it: we scale it later
    df['vector_full'] = df[f'vector_{depth}'] + df['vector_full']
    df['vector_new'] = df[f'vector_{depth}']

exp = 'exp1'

# save records
df.to_parquet(f'{exp}/df_hierarchical_vector.parquet', index=None)

### computing msc for every sample in the dataset

In [5]:
exp = 'exp1'
depth = 1
df = pd.read_parquet(f'{exp}/df_hierarchical_vector.parquet')
df_new_clean = df[['class_1', 'vector_1']].drop_duplicates(f'class_{depth}')
n_classes = len(df_new_clean.class_1.unique())
new_vector_length = len(df.vector_1[0])
selection_method = 'precise' # 'approximate' 'precise'

seq = dict()
for key_A in [*df_new_clean['class_1']]:
    nbrs = NearestNeighbors(n_neighbors=len(df_new_clean), metric='euclidean').fit(df_new_clean['vector_1'].values.tolist())
    distances, indices = nbrs.kneighbors(df_new_clean[df_new_clean['class_1']==key_A]['vector_1'].values.tolist())
    seq[key_A] = [*df_new_clean['class_1'].iloc[indices[0]].values]

t = list()
total = list()
for key_A in [*seq.keys()]:
    if key_A not in ['m']: #[*df_total[0].unique()][:-1]:

        df = pd.read_parquet(f'{exp}/df_hierarchical_vector.parquet')

        for _ in seq[key_A]:
            # find the path from key_A to all the following keys in order of distance
            sequence_ = list()
            for index in range(len(seq[key_A])-1):
                sequence_.append([seq[key_A][index], seq[key_A][index+1]])
            sequence_

        for s in sequence_:
            key_B = s[0]
            key_C = s[1]

            if selection_method == 'precise':
                # we take into account ALL THE OTHER POINTS: heavy computations required
                nbrs_B = NearestNeighbors(n_neighbors=len(df[df['class_1']==key_B]), metric='euclidean').fit(df[df['class_1']==key_B]['vector_full'].values.tolist())
                nbrs_C = NearestNeighbors(n_neighbors=len(df[df['class_1']==key_C]), metric='euclidean').fit(df[df['class_1']==key_C]['vector_full'].values.tolist())
            elif selection_method == 'approximate':
                # we apply k-means to every class
                # df_sample = 
                pass
                
            counter = 0
            for k in df[df['class_1']==key_A].reset_index().values.tolist():
                point_A = k[0] # index is the absolute one
                point_A_vector = k[4]

                distances_B, indices_B = nbrs_B.kneighbors([point_A_vector])
                distances_C, indices_C = nbrs_C.kneighbors([point_A_vector])

                # given point_A, find the furthest in cluster_B
                point_B = indices_B[0][-1]
                point_B_vector = df[df['class_1']==key_B]['vector_full'].iloc[point_B].tolist()
                longest_vectors = (point_A_vector, point_B_vector)

                # given point_A, find the shortest in cluster_C
                point_C = indices_C[0][0]
                point_C_vector = df[df['class_1']==key_C]['vector_full'].iloc[point_C].tolist()
                shortest_vectors = (point_A_vector, point_C_vector)

                x = triangular_extension(
                    point_A_vector, 
                    point_B_vector,
                    point_C_vector, 
                    concat_vector_length=new_vector_length
                )

                total.append([key_A, key_B, key_C, point_A, x])
                t.append([point_A_vector, point_B_vector, point_C_vector])

                # print(point_A_vector, point_B_vector, point_C_vector)
                counter += 1           

            # print insights
            df_total = pd.DataFrame(total)
            df_total.columns = ['A', 'B', 'C', 'index', 'minimum_scale_coefficient']
            minimum_scale_coefficient_key_B = df_total[(df_total['A']==key_A) & (df_total['B']==key_B)]['minimum_scale_coefficient'].max()
            print('A:', key_A, '\tB:', key_B, '\tC:', key_C, '\t**', minimum_scale_coefficient_key_B)

df_total.to_parquet(f'{exp}/total.csv', index=None)

A: 07 	B: 07 	C: 05 	** 1.045578633853104
A: 07 	B: 05 	C: 01 	** 2.703447207283985
A: 07 	B: 01 	C: 00 	** 6.255404885398754
A: 07 	B: 00 	C: 03 	** 2.351017413767862
A: 07 	B: 03 	C: 04 	** 2.818713063713374
A: 07 	B: 04 	C: 02 	** 3.704776385354303
A: 07 	B: 02 	C: 06 	** 3.178329666477664
A: 01 	B: 01 	C: 07 	** 0.976130536897291
A: 01 	B: 07 	C: 03 	** 2.398549853704585
A: 01 	B: 03 	C: 06 	** 6.895294989035782
A: 01 	B: 06 	C: 00 	** 3.3447132120891934
A: 01 	B: 00 	C: 04 	** 3.5110103012352525
A: 01 	B: 04 	C: 05 	** 6.235429747408428
A: 01 	B: 05 	C: 02 	** 4.331186783789231
A: 03 	B: 03 	C: 02 	** 0.9447831214171218
A: 03 	B: 02 	C: 01 	** 2.5588594553733794
A: 03 	B: 01 	C: 07 	** 5.664591799747541
A: 03 	B: 07 	C: 00 	** 4.515299627196445
A: 03 	B: 00 	C: 05 	** 2.6670200370685726
A: 03 	B: 05 	C: 06 	** 3.86121335247303
A: 03 	B: 06 	C: 04 	** 3.532013526015351
A: 02 	B: 02 	C: 03 	** 0.9091300475256696
A: 02 	B: 03 	C: 04 	** 2.0925851684459627
A: 02 	B: 04 	C: 00 	** 3.24

In [ ]:
df_total

### testing a single sample

In [6]:
import sys

df = pd.read_parquet(f'{exp}/df_hierarchical_vector.parquet') #we reset the vectors so they do not kee scaling per iteration

# test msc of single sample
sample_id = 3
msc = get_msc(df_total, sample_id=sample_id, verbose=False)
# without adding the 0.001, it fails because of function approximation

# by a tiny decrease in x, the constraint should not be valida anymore
test_amount = 0.00001
class_n = [*df_total[df_total['index']==sample_id]['A']][0]
test_knn(df, class_n=class_n, minimum_scale_coefficient=msc, test_amount=test_amount, sample_id=sample_id, verbose=False)

True

### counting geometrical impossibilities, if any

In [7]:
# count geometrical impossibilities
len(df_total[df_total['minimum_scale_coefficient']==-1])

0

### testing the entire dataset by adding/removing epsilon (test_score)

In [9]:
from tqdm import tqdm

for key_A in [*df_total['A'].unique()]:
    for sample_index in tqdm(range(len(df_total[df_total['A']==key_A]['index'])), desc=f'class: {key_A}'):
        sample_id = df_total[df_total['A']==key_A]['index'].iloc[sample_index]
        msc = get_msc(df_total, sample_id=sample_id, verbose=False)
        test = test_knn(
            df, 
            class_n=key_A, 
            minimum_scale_coefficient=msc, 
            test_amount=test_amount,
            sample_id=sample_id, 
            verbose=False
        )
        if test == False : break

class: 02:  27%|██▋       | 167/609 [00:12<00:33, 13.34it/s]


KeyboardInterrupt: 

### computing merge function

In [112]:
import plotly.graph_objects as go

def plot_series(series):
    x_values = [item[0] for item in series]
    y_values = [item[1] for item in series]
    
    fig = go.Figure(data=go.Scatter(x=x_values, y=y_values, mode='lines'))
    fig.update_layout(
        title='HVC-function',
        xaxis_title='msc',
        yaxis_title='contaminations',
        template='plotly_dark'
    )
    fig.show()

# count contaminationsA
fn_passing = list()
passing_score = 0
while passing_score < df_total['minimum_scale_coefficient'].max():
    passing_score += 0.05
    df_total['pass'] = df_total['minimum_scale_coefficient'].apply(lambda x : x > passing_score)
    merge_intensity = len(df_total[df_total['pass']]) #/len(df_total)
    fn_passing.append([passing_score, merge_intensity])

plot_series(fn_passing)

By inverting the function we can input the desired number of contaminations, and get the msc required to obtain it

In [116]:
import plotly.graph_objects as go

def plot_series(series):
    x_values = [item[1] for item in series]
    y_values = [item[0] for item in series]
    
    fig = go.Figure(data=go.Scatter(x=x_values, y=y_values, mode='lines'))
    fig.update_layout(
        title='HVC-function',
        xaxis_title='contaminations',
        yaxis_title='msc',
        template='plotly_dark'
    )
    fig.show()

# count contaminationsA
fn_passing = list()
passing_score = 0
while passing_score < df_total['minimum_scale_coefficient'].max():
    passing_score += 0.05
    df_total['pass'] = df_total['minimum_scale_coefficient'].apply(lambda x : x > passing_score)
    merge_intensity = len(df_total[df_total['pass']]) #/len(df_total)
    fn_passing.append([passing_score, merge_intensity])

plot_series(fn_passing)

After normalization, we can simply input the desired merge intensity and get the msc required to obtain it

In [114]:
def plot_series(series):
    x_values = [item[0] for item in series]
    y_values = [item[1] for item in series]
    
    fig = go.Figure(data=go.Scatter(x=x_values, y=y_values, mode='lines'))
    fig.update_layout(
        title='HVC-function',
        xaxis_title='merge_intensity',
        yaxis_title='msc',
        template='plotly_dark'
    )
    fig.show()

# count contaminations
fn_passing = list()
passing_score = 0
while passing_score < df_total['minimum_scale_coefficient'].max():
    passing_score += 0.05
    df_total['pass'] = df_total['minimum_scale_coefficient'].apply(lambda x : x > passing_score)
    merge_intensity = len(df_total[df_total['pass']])/len(df_total)
    fn_passing.append([merge_intensity, passing_score])

plot_series(fn_passing)